Neste ex. se usa a biblioteca langchain para importar um moldeo de linguagem da openai. Configura-se o modelo com uma chave de api e cria-se
uma chain para processar o prompt "Qual a capital da frança?", com a resposta sendo exibida na saída.

In [2]:
!pip install langchain

/bin/bash: /home/koiti/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [18]:
# Importando a biblioteca Langchain e o modelo de linguagem da OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

In [19]:
load_dotenv()                     # procura .env na pasta atual; passe um caminho se estiver fora
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 2. Instancie o modelo de chat:
llm = ChatOpenAI(
    openai_api_key= OPENAI_API_KEY,
    model_name="gpt-3.5-turbo",
    temperature=0.7
)



In [20]:
# 3. Crie o PromptTemplate e a Chain:
prompt = PromptTemplate(
    input_variables=["question"],
    template="{question}"
)
chain = LLMChain(llm=llm, prompt=prompt)


In [6]:
# 4. Rode:
response = chain.run({"question": "Qual é a capital da França?"})
print(response)

/tmp/ipykernel_16324/338267425.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run({"question": "Qual é a capital da França?"})


A capital da França é Paris.


exemplo de uso de prompt template

In [7]:
from langchain.prompts import PromptTemplate

# Criando um template de prompt
template = PromptTemplate.from_template("Qual é a capital de {country}?")

# Utilizando o template com o modelo de linguagem
prompt = template.format(country="Brasil")
response = chain.run(prompt)

print(response)

A capital do Brasil é Brasília.


 Por exemplo, imagine que você queira montar um sistema de perguntas e respostas sobre contratos jurídicos armazenados em PDF. Primeiro você carregaria os PDFs com um UnstructuredPDFLoader ou PyPDFLoader, depois dividiria o texto em trechos com RecursiveCharacterTextSplitter, criaria embeddings via OpenAIEmbeddings e indexaria tudo em uma estrutura FAISS. Aí basta ligar um RetrievalQA para que, a cada pergunta, o LangChain recupere os trechos mais relevantes e alimente o LLM num chain de “map-reduce” ou “refine”.

In [8]:
"""
Pipeline completo de RAG em LangChain que:

1. Faz download automático de um PDF público (Declaração Universal dos Direitos Humanos).
2. Carrega e divide o PDF em chunks.
3. Gera embeddings com OpenAI.
4. Armazena os vetores em FAISS.
5. Usa um modelo ChatGPT (gpt-3.5-turbo) para responder perguntas a partir do retriever.

Pré-requisitos
--------------
pip install langchain langchain-community \
            openai python-dotenv \
            faiss-cpu tiktoken \
            requests
# se preferir CUDA, troque 'faiss-cpu' por 'faiss-gpu' e adicione cudatoolkit compatível.

Crie um arquivo .env no mesmo diretório contendo:
OPENAI_API_KEY=sk-sua_nova_chave
Não compartilhe esse arquivo nem faça commit no Git.
"""

import os
import requests
from dotenv import load_dotenv
from pathlib import Path

# LangChain – loaders, splitters, embeddings, vectorstores, LLM, QA chain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain_community.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

# --------------------------------------------------
# 0. Configuração de chaves e caminhos
# --------------------------------------------------
# Se o .env está no diretório atual, ficamos; caso contrário subimos um nível (ex.: notebooks em exercicios/)
ROOT_DIR = Path.cwd() if (Path.cwd() / ".env").exists() else Path.cwd().parent
load_dotenv(ROOT_DIR / ".env")      # carrega OPENAI_API_KEY sem expor no código

PDF_URL  = "https://www.un.org/en/udhrbook/pdf/udhr_booklet_en_web.pdf"
PDF_PATH = os.path.expanduser("~/Downloads/llm/udhr_booklet_en_web.pdf")

# --------------------------------------------------
# 1. Download do PDF, se necessário
# --------------------------------------------------
if not os.path.exists(PDF_PATH):
    print("Baixando PDF...")
    resp = requests.get(PDF_URL, timeout=30)
    resp.raise_for_status()
    os.makedirs(os.path.dirname(PDF_PATH), exist_ok=True)
    with open(PDF_PATH, "wb") as f:
        f.write(resp.content)
    print(f"PDF salvo em {PDF_PATH}")
else:
    print(f"PDF já existe em {PDF_PATH}")

# --------------------------------------------------
# 2. Leitura e divisão em trechos
# --------------------------------------------------
loader   = PyPDFLoader(PDF_PATH)
docs     = loader.load()  # lista de Document
splitter = RecursiveCharacterTextSplitter(chunk_size=1_000, chunk_overlap=200)
chunks   = splitter.split_documents(docs)

print(f"Total de chunks: {len(chunks)}")

# --------------------------------------------------
# 3. Embeddings + Faiss
# --------------------------------------------------
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(chunks, embeddings)
retriever   = vectorstore.as_retriever(search_kwargs={"k": 4})

# --------------------------------------------------
# 4. Modelo de linguagem (chat)
# --------------------------------------------------
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.1
)

# --------------------------------------------------
# 5. Cadeia de QA por refinamento
# --------------------------------------------------
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine",        # ou 'stuff' / 'map_reduce'
    retriever=retriever,
    return_source_documents=True
)

# --------------------------------------------------
# 6. Exemplo de consulta
# --------------------------------------------------
query = "Qual artigo trata da liberdade de expressão?"
result = qa_chain(query)

print("\n=== Resposta ===")
print(result["result"])

print("\n=== Trechos de suporte ===")
for doc in result["source_documents"]:
    print(f"- [{doc.metadata.get('page', 'desconhecida')}] {doc.page_content[:200]}...")


PDF já existe em /home/koiti/Downloads/llm/udhr_booklet_en_web.pdf
Total de chunks: 40


/tmp/ipykernel_16324/2193829548.py:73: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()
/tmp/ipykernel_16324/2193829548.py:99: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain(query)



=== Resposta ===
O artigo que trata da liberdade de expressão é o Artigo 19 da Declaração Universal dos Direitos Humanos. Ele afirma que "Todo ser humano tem direito à liberdade de opinião e expressão; esse direito inclui a liberdade de, sem interferência, ter opiniões e de procurar, receber e transmitir informações e ideias por quaisquer meios e independentemente de fronteiras." Além disso, o Artigo 20 da mesma declaração garante o direito à liberdade de reunião pacífica e associação. Esses direitos estão intrinsecamente ligados à liberdade, segurança e dignidade de cada indivíduo, conforme estabelecido no Artigo 3 da Declaração Universal dos Direitos Humanos. O Artigo 18 também é relevante, pois garante o direito à liberdade de pensamento, consciência e religião, incluindo a liberdade de mudar de religião ou crença e de manifestar essa religião ou crença em público ou privado, individualmente ou em comunidade.

=== Trechos de suporte ===
- [48] | Universal Declaration of Human Right

Este processo pode ser feito usando o chromdb em vez do Faiss tambem, como indicado na celula abaixo

In [9]:
import os
import requests
from dotenv import load_dotenv
from pathlib import Path

# LangChain – loaders, splitters, embeddings, vectorstores, LLM, QA chain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_chroma import Chroma                  # ← troca o import
from langchain_community.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

# --------------------------------------------------
# 0. Configuração de chaves e caminhos
# --------------------------------------------------
ROOT_DIR = Path.cwd() if (Path.cwd() / ".env").exists() else Path.cwd().parent
load_dotenv(ROOT_DIR / ".env")      # lê OPENAI_API_KEY do .env

PDF_URL  = "https://www.un.org/en/udhrbook/pdf/udhr_booklet_en_web.pdf"
PDF_PATH = os.path.expanduser("~/Downloads/llm/udhr_booklet_en_web.pdf")

# --------------------------------------------------
# 1. Download do PDF
# --------------------------------------------------
if not os.path.exists(PDF_PATH):
    resp = requests.get(PDF_URL, timeout=30)
    resp.raise_for_status()
    os.makedirs(os.path.dirname(PDF_PATH), exist_ok=True)
    with open(PDF_PATH, "wb") as f:
        f.write(resp.content)

# --------------------------------------------------
# 2. Carregamento e split
# --------------------------------------------------
loader   = PyPDFLoader(PDF_PATH)
docs     = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=1_000, chunk_overlap=200)
chunks   = splitter.split_documents(docs)

# --------------------------------------------------
# 3. Embeddings + Chroma
# --------------------------------------------------
embeddings = OpenAIEmbeddings()

# Se quiser persistir no disco entre execuções, indique persist_directory;
# se deixar None (ou omitir), tudo roda apenas em memória.
VECTOR_DIR = os.path.expanduser("~/Downloads/llm/chroma_udhr")
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    collection_name="udhr",
    persist_directory=VECTOR_DIR               # ← opcional; comenta se não quiser persistir
)                                              # ← criação do índice

retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

# --------------------------------------------------
# 4. Modelo de linguagem
# --------------------------------------------------
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1)

# --------------------------------------------------
# 5. Cadeia de QA
# --------------------------------------------------
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine",
    retriever=retriever,
    return_source_documents=True,
)

# --------------------------------------------------
# 6. Exemplo de consulta
# --------------------------------------------------
query = "Qual artigo trata da liberdade de expressão?"
result = qa_chain(query)

print("\n=== Resposta ===")
print(result["result"])

print("\n=== Trechos de suporte ===")
for doc in result["source_documents"]:
    print(f"- [pág. {doc.metadata.get('page', 'desconhecida')}] {doc.page_content[:200]}...")



=== Resposta ===
O Artigo 19 da Declaração Universal dos Direitos Humanos trata da liberdade de expressão, garantindo que todo ser humano tem o direito à liberdade de opinião e expressão, incluindo a liberdade de buscar, receber e transmitir informações e ideias sem interferência e independentemente de fronteiras. Este direito está intrinsecamente ligado à liberdade de pensamento, consciência e religião, conforme estabelecido no Artigo 18 da mesma declaração. Ambos os artigos são fundamentais para a proteção dos direitos humanos e da democracia.

=== Trechos de suporte ===
- [pág. 48] | Universal Declaration of Human Rights | 40
ArticleUNITED
NATIONS
Everyone has the right to freedom 
of opinion and expression; this right 
includes freedom to hold opinions with -
out interference a...
- [pág. 50] | Universal Declaration of Human Rights | 42
ArticleUNITED
NATIONS
(1) Everyone has the right to freedom    
of peaceful assembly and association. 
(2) No one may be compelled to belong 
to a

Para diálogos mais ricos, você pode adicionar memória de contexto. Usar ConversationBufferMemory faz com que cada troca seja empilhada e reapresentada ao LLM; usar ConversationSummaryMemory resume a conversa antiga para economizar tokens. Com isso, o assistente “lembra” de detalhes sem precisar reenviar tudo:

In [11]:
"""
Pipeline Conversational RAG — Declaração Universal dos Direitos Humanos

1. Baixa o PDF da ONU (se ainda não existir).
2. Constrói FAISS + embeddings OpenAI.
3. Usa ConversationalRetrievalChain com memória de buffer.

Requisitos
----------
pip install langchain langchain-community \
            openai python-dotenv \
            faiss-cpu tiktoken \
            requests
"""

import os
import requests
from dotenv import load_dotenv
from pathlib import Path

# LangChain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain_community.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

# --------------------------------------------------
# 0. Ambiente
# --------------------------------------------------
# Se o .env está no diretório atual, ficamos; caso contrário subimos um nível (ex.: notebooks em exercicios/)
ROOT_DIR = Path.cwd() if (Path.cwd() / ".env").exists() else Path.cwd().parent
load_dotenv(ROOT_DIR / ".env")      # carrega OPENAI_API_KEY sem expor no código

PDF_URL  = "https://www.un.org/en/udhrbook/pdf/udhr_booklet_en_web.pdf"
PDF_PATH = os.path.expanduser("~/Downloads/llm/udhr_booklet_en_web.pdf")

# --------------------------------------------------
# 1. Download do PDF
# --------------------------------------------------
if not os.path.exists(PDF_PATH):
    print("Baixando PDF...")
    r = requests.get(PDF_URL, timeout=30)
    r.raise_for_status()
    os.makedirs(os.path.dirname(PDF_PATH), exist_ok=True)
    with open(PDF_PATH, "wb") as fp:
        fp.write(r.content)
    print(f"PDF salvo em {PDF_PATH}")
else:
    print(f"PDF já existe em {PDF_PATH}")

# --------------------------------------------------
# 2. Carrega e divide
# --------------------------------------------------
loader   = PyPDFLoader(PDF_PATH)
docs     = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=1_000, chunk_overlap=200)
chunks   = splitter.split_documents(docs)
print(f"Total de chunks: {len(chunks)}")

# --------------------------------------------------
# 3. Embeddings + FAISS
# --------------------------------------------------
embeddings  = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(chunks, embeddings)
retriever   = vectorstore.as_retriever(search_kwargs={"k": 4})

# --------------------------------------------------
# 4. LLM e memória
# --------------------------------------------------
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.2
)

memory = ConversationBufferMemory(
    memory_key="chat_history",
    output_key="answer",      
    return_messages=True
)

chat_rag = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    return_source_documents=True
)

# --------------------------------------------------
# 5. Diálogo de teste
# --------------------------------------------------
print("\nPergunta 1:")
resp1 = chat_rag({"question": "Quem é o responsável por esses direitos?"})
print(resp1["answer"])

print("\nPergunta 2:")
resp2 = chat_rag({"question": "E qual artigo trata da liberdade de expressão?"})
print(resp2["answer"])

# --------------------------------------------------
# 6. Histórico e fontes
# --------------------------------------------------
print("\n=== Histórico ===")
for msg in memory.chat_memory.messages:
    role = "User" if msg.type == "human" else "AI"
    print(f"{role}: {msg.content[:80]}...")

print("\n=== Fontes ===")
for doc in resp2["source_documents"]:
    page = doc.metadata.get("page", "desconhecida")
    snippet = doc.page_content.replace("\n", " ")[:120]
    print(f"[p.{page}] {snippet}…")


PDF já existe em /home/koiti/Downloads/llm/udhr_booklet_en_web.pdf
Total de chunks: 40

Pergunta 1:
Os direitos mencionados nos trechos fornecidos são parte da Declaração Universal dos Direitos Humanos, adotada e proclamada pela Assembleia Geral das Nações Unidas em 1948.

Pergunta 2:
O artigo da Declaração Universal dos Direitos Humanos que trata da liberdade de expressão é o Artigo 19. Ele afirma que "Todo ser humano tem direito à liberdade de opinião e expressão; esse direito inclui a liberdade de, sem interferência, ter opiniões e de procurar, receber e transmitir informações e ideias por quaisquer meios e independentemente de fronteiras."

=== Histórico ===
User: Quem é o responsável por esses direitos?...
AI: Os direitos mencionados nos trechos fornecidos são parte da Declaração Universal...
User: E qual artigo trata da liberdade de expressão?...
AI: O artigo da Declaração Universal dos Direitos Humanos que trata da liberdade de ...

=== Fontes ===
[p.48] | Universal Declaration 

In [12]:
"""
Pipeline Conversational RAG — Declaração Universal dos Direitos Humanos
versão ChromaDB + OpenAI + memória de buffer
"""

import os
import requests
from dotenv import load_dotenv
from pathlib import Path

# LangChain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_chroma import Chroma                   # ← novo import
from langchain_community.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

# --------------------------------------------------
# 0. Ambiente
# --------------------------------------------------
ROOT_DIR = Path.cwd() if (Path.cwd() / ".env").exists() else Path.cwd().parent
load_dotenv(ROOT_DIR / ".env")                 # lê OPENAI_API_KEY

PDF_URL  = "https://www.un.org/en/udhrbook/pdf/udhr_booklet_en_web.pdf"
PDF_PATH = os.path.expanduser("~/Downloads/llm/udhr_booklet_en_web.pdf")

# --------------------------------------------------
# 1. Download do PDF
# --------------------------------------------------
if not os.path.exists(PDF_PATH):
    print("Baixando PDF…")
    r = requests.get(PDF_URL, timeout=30)
    r.raise_for_status()
    os.makedirs(os.path.dirname(PDF_PATH), exist_ok=True)
    with open(PDF_PATH, "wb") as fp:
        fp.write(r.content)

# --------------------------------------------------
# 2. Carrega e divide
# --------------------------------------------------
loader   = PyPDFLoader(PDF_PATH)
docs     = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=1_000, chunk_overlap=200)
chunks   = splitter.split_documents(docs)
print(f"Total de chunks: {len(chunks)}")

# --------------------------------------------------
# 3. Embeddings + Chroma
# --------------------------------------------------
embeddings = OpenAIEmbeddings()

# Se quiser reutilizar o índice entre execuções, defina persist_directory
# (o caminho abaixo é só um exemplo; remova o parâmetro para rodar em memória)
VECTOR_DIR = os.path.expanduser("~/Downloads/llm/chroma_udhr")

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    collection_name="udhr",
    persist_directory=VECTOR_DIR          # comente esta linha se não precisar persistir
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

# --------------------------------------------------
# 4. LLM e memória de conversa
# --------------------------------------------------
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.2)

memory = ConversationBufferMemory(
    memory_key="chat_history",
    output_key="answer",
    return_messages=True
)

chat_rag = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    return_source_documents=True
)

# --------------------------------------------------
# 5. Diálogo de teste
# --------------------------------------------------
print("\nPergunta 1:")
resp1 = chat_rag({"question": "Quem é o responsável por esses direitos?"})
print(resp1["answer"])

print("\nPergunta 2:")
resp2 = chat_rag({"question": "E qual artigo trata da liberdade de expressão?"})
print(resp2["answer"])

# --------------------------------------------------
# 6. Histórico e fontes
# --------------------------------------------------
print("\n=== Histórico ===")
for msg in memory.chat_memory.messages:
    role = "User" if msg.type == "human" else "AI"
    print(f"{role}: {msg.content[:80]}…")

print("\n=== Fontes ===")
for doc in resp2["source_documents"]:
    page = doc.metadata.get("page", "desconhecida")
    snippet = doc.page_content.replace("\n", " ")[:120]
    print(f"[p.{page}] {snippet}…")


Total de chunks: 40

Pergunta 1:
Os direitos mencionados nos trechos fornecidos são garantidos pela Declaração Universal dos Direitos Humanos, que é um documento adotado pela Assembleia Geral das Nações Unidas.

Pergunta 2:
O artigo da Declaração Universal dos Direitos Humanos que trata da liberdade de expressão é o Artigo 19. Ele afirma que "Todo ser humano tem direito à liberdade de opinião e expressão".

=== Histórico ===
User: Quem é o responsável por esses direitos?…
AI: Os direitos mencionados nos trechos fornecidos são garantidos pela Declaração Un…
User: E qual artigo trata da liberdade de expressão?…
AI: O artigo da Declaração Universal dos Direitos Humanos que trata da liberdade de …

=== Fontes ===
[p.48] | Universal Declaration of Human Rights | 40 ArticleUNITED NATIONS Everyone has the right to freedom  of opinion and exp…
[p.48] | Universal Declaration of Human Rights | 40 ArticleUNITED NATIONS Everyone has the right to freedom  of opinion and exp…
[p.50] | Universal Decl

Se o seu caso exigir decisões dinâmicas – por exemplo, primeiro buscar na internet, depois consultar uma base de dados interna, depois gerar um relatório – você pode criar um agente que receba várias “tools” (funções Python, chamadas HTTP, queries SQL) e deixe que o modelo opine qual deve chamar a cada passo. No modo ZERO_SHOT_REACT_DESCRIPTION, o LangChain cria um prompt invisível que enumera os tools disponíveis e suas descrições. O modelo de linguagem “raciocina” passo a passo (cotação Thought/Action/Observation) e escolhe o tool cujo nome e descrição melhor se encaixam no que precisa fazer para responder.

In [17]:
import os, requests, sqlite3          # sqlite = exemplo de BD local
from dotenv import load_dotenv
from pathlib import Path

# LangChain – mesmo fluxo de antes
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.agents import Tool, initialize_agent, AgentType

# --------------------------------------------------
# 0. Ambiente e PDF da Declaração
# --------------------------------------------------
ROOT_DIR = Path.cwd() if (Path.cwd() / ".env").exists() else Path.cwd().parent
load_dotenv(ROOT_DIR / ".env")

PDF_URL  = "https://www.un.org/en/udhrbook/pdf/udhr_booklet_en_web.pdf"
PDF_PATH = os.path.expanduser("~/Downloads/llm/udhr_booklet_en_web.pdf")

if not os.path.exists(PDF_PATH):
    r = requests.get(PDF_URL, timeout=30); r.raise_for_status()
    os.makedirs(os.path.dirname(PDF_PATH), exist_ok=True)
    with open(PDF_PATH, "wb") as fp: fp.write(r.content)

# --------------------------------------------------
# 1. Indexa a UDHR
# --------------------------------------------------
loader   = PyPDFLoader(PDF_PATH)
docs     = loader.load()
chunks   = RecursiveCharacterTextSplitter(
              chunk_size=1_000, chunk_overlap=200).split_documents(docs)
vectorstore = FAISS.from_documents(chunks, OpenAIEmbeddings())
retriever   = vectorstore.as_retriever(search_kwargs={"k": 4})
qa_chain_udhr = RetrievalQA.from_chain_type(
                   llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1),
                   chain_type="refine",
                   retriever=retriever,
                   return_source_documents=False)

def udhr_tool(q: str) -> str:
    return qa_chain_udhr.run(q)

# --------------------------------------------------
# 2. Tool: busca web (exemplo mínimo com DuckDuckGo JSON)
#    → troque pela sua API/SerpAPI preferida
# --------------------------------------------------
def search_web(query: str) -> str:
    api = "https://duckduckgo.com/js/spice/search"
    resp = requests.get(api, params={"q": query, "format": "json"}, timeout=10)
    resp.raise_for_status()
    # resposta simplificada: primeiro snippet encontrado
    data = resp.json()
    if data.get("Results"):
        return data["Results"][0]["Text"]
    return "Nenhum resultado encontrado."

# --------------------------------------------------
# 3. Tool: consulta SQL (sqlite apenas para ilustrar)
# --------------------------------------------------
DB_PATH = os.path.expanduser("~/Downloads/llm/contracts.db")
conn = sqlite3.connect(DB_PATH)

def query_sql(sql: str) -> str:
    try:
        cur = conn.execute(sql)
        rows = cur.fetchall()
        return f"{len(rows)} linhas → {rows[:3]}…"  # mostra no máximo 3
    except Exception as e:
        return f"Erro SQL: {e}"

# --------------------------------------------------
# 4. Agente com três tools
# --------------------------------------------------
tools = [
    Tool(name="udhr_search",
         description="Responde perguntas sobre a Declaração Universal dos Direitos Humanos.",
         func=udhr_tool),
    Tool(name="web_search",
         description="Pesquisa informações públicas recentes na web.",
         func=search_web),
    Tool(name="contract_db",
         description="Consulta cláusulas e multas no banco de dados de contratos; aceita SQL SELECT.",
         func=query_sql),
]

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# --------------------------------------------------
# 5. Exemplos de perguntas que exigem ferramentas diferentes
# --------------------------------------------------
# a) Precisa da UDHR + SQL
q1 = ("Compare o Artigo 19 da Declaração Universal dos Direitos Humanos "
      "com a cláusula Penalty_LateFees do contrato 42 e diga se há conflito.")
print("\n--- Resposta 1 ---")
print(agent.run(q1))

# b) Precisa de pesquisa web (resolução recente da ONU)
q2 = ("Quais países votaram contra a resolução da ONU de 2023 sobre liberdade de expressão? "
      "Cite a fonte resumidamente.")
print("\n--- Resposta 2 ---")
print(agent.run(q2))





--- Resposta 1 ---


> Entering new AgentExecutor chain...
I need to compare Article 19 of the Universal Declaration of Human Rights with the clause Penalty_LateFees in contract 42 to see if there is a conflict.
Action: udhr_search
Action Input: "Article 19 Universal Declaration of Human Rights"
Observation: The additional context provided does not directly relate to Article 19 of the Universal Declaration of Human Rights, which focuses on freedom of opinion and expression. Therefore, the original answer remains relevant and does not need to be refined with the new information.
Thought:I need to search for the specific clause Penalty_LateFees in contract 42 to compare with Article 19 of the UDHR.
Action: contract_db
Action Input: "SELECT Penalty_LateFees FROM Contract WHERE ContractNumber = 42"
Observation: Erro SQL: no such table: Contract
Thought:I need to correct the SQL query to ensure it retrieves the clause Penalty_LateFees from contract 42.
Action: contract_db
Action Input: "SE